In [3]:
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr

# --- 1. Math Functions (Same as before) ---

def sigmoid_func(x):
    """Calculate standard sigmoid function"""
    # Prevent overflow
    return np.where(x >= 0,
                    1 / (1 + np.exp(-x)),
                    np.exp(x) / (1 + np.exp(x)))

def logistic_growth(t, M, r, N0):
    """Calculate logistic growth analytical solution"""
    # Edge cases
    if N0 == 0: return np.zeros_like(t)
    if N0 >= M: return np.full_like(t, M)

    # Calculate constant C based on integration
    C = (M - N0) / N0

    # Calculate denominator safely
    denominator = 1 + C * np.exp(-r * t)

    return M / denominator


# --- 2. Plotting Function for Gradio ---

def plot_comparison(M, r, N0, T_max):
    """
    Plotting function for Gradio interface.
    Inputs: Sliders (M, r, N0, T_max)
    Output: matplotlib figure object
    """
    # --- Data Preparation ---
    # (1) x-axis for Standard Sigmoid
    x_sigmoid = np.linspace(-10, 10, 400)

    # (2) t-axis for Population Model
    t_pop = np.linspace(0, T_max, 400)

    # Calculate Population
    y_pop = logistic_growth(t_pop, M, r, N0)

    # --- Plotting ---
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # [Left Plot] Standard Sigmoid Function f(x)
    axes[0].plot(x_sigmoid, sigmoid_func(x_sigmoid), color='black', linewidth=3, label=r'$f(x) = \frac{1}{1+e^{-x}}$')
    axes[0].set_title("1. Standard Sigmoid Function", fontsize=14, fontweight='bold')
    axes[0].set_xlabel("Input (x)")
    axes[0].set_ylabel("Output (f(x))")
    axes[0].axhline(0, color='gray', linestyle='--', linewidth=0.8)
    axes[0].axhline(0.5, color='blue', linestyle=':', label="Midpoint (0.5)")
    axes[0].axhline(1, color='red', linestyle='--', label="Max Limit (1.0)")
    axes[0].axvline(0, color='gray', linestyle='--', linewidth=0.8)
    axes[0].set_ylim(-0.1, 1.1)
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)

    # [Right Plot] Logistic Population Model N(t)
    axes[1].plot(t_pop, y_pop, color='green', linewidth=3, label=f'Model Curve\n(M={M}, r={r:.2f})')
    axes[1].set_title("2. Logistic Population Model N(t)", fontsize=14, fontweight='bold')
    axes[1].set_xlabel("Time (t)")
    axes[1].set_ylabel("Population (N)")

    # Carrying Capacity (M) Line
    axes[1].axhline(M, color='red', linestyle='--', linewidth=1.5, label=f'Carrying Capacity (M={M})')
    # Initial Population (N0) Point
    axes[1].plot(0, N0, 'bo', markersize=8, label=f'Initial Pop. (N0={N0})')

    axes[1].legend(fontsize=11)
    axes[1].grid(True, alpha=0.3)

    # Dynamic Y-axis limit
    axes[1].set_ylim(0, M * 1.2)
    axes[1].set_xlim(0, T_max)

    plt.tight_layout()

    return fig


# --- 3. Gradio Interface ---

# Input Components (Sliders)
inputs = [
    gr.Slider(minimum=100, maximum=5000, value=1000, step=100, label="Carrying Capacity (M) - Adjusts Max Height"),
    gr.Slider(minimum=0.01, maximum=1.0, value=0.2, step=0.01, label="Growth Rate (r) - Adjusts Steepness"),
    gr.Slider(minimum=1, maximum=500, value=10, step=1, label="Initial Population (N0) - Adjusts Starting Point"),
    gr.Slider(minimum=50, maximum=300, value=100, step=10, label="Simulation Time (T_max) - X-axis Range")
]

# Output Component
output = gr.Plot(label="Result Plot")

# Interface Definition
demo = gr.Interface(
    fn=plot_comparison,
    inputs=inputs,
    outputs=output,
    title="Logistic Growth Model Simulator",
    description="""
    ### Explore the relationship between the Standard Sigmoid Function and the Logistic Population Model.
    The left plot shows the fixed mathematical function. The right plot shows the real-world model controlled by the sliders below.

    * **M (Carrying Capacity):** Sets the 'ceiling' or maximum population size (Graph Height).
    * **r (Growth Rate):** Controls how fast the population reaches M (Graph Slope/Steepness).
    * **N0 (Initial Population):** Sets the starting population at time t=0.
    """,
    live=True
)

# Launch
if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e4a478e8045d16842e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 시그모이드 함수(Sigmoid Function)의 의미
- 로지스틱 성장과 질병 확산 모델에서 나타나는 전형적인 **S자 곡선(S-curve)**의 수식  
  $$\[
  f(x)=\frac{1}{1+e^{-x}}
  \]$$
- 특징  
  - 초기에 증가가 매우 느림  
  - 중간 구간에서 급격히 증가  
  - 최종적으로 상한값(1 또는 환경수용력 \(M\))에 수렴  
- 질병 모델에서의 역할  
  - 감염될 사람이 거의 남지 않거나  
  - 집단면역이 형성되면  
    → 감염 확산이 멈추는 이상적인 모습을 표현
